# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from google.colab import files
import os
import warnings
import pickle
import json
from google.colab import drive

warnings.filterwarnings("ignore")
%matplotlib inline

# Mount drive, Load config

In [ ]:
PROJECT_PATH = "/content/drive/MyDrive/Projects/GitHub/Spotify/"
CONFIG_FILE = f"{PROJECT_PATH}src/config.json"

In [ ]:
DATA_FILE   = "/content/spotify_data/spotify_songs.csv"

In [ ]:
drive.mount('/content/drive')

with open(CONFIG_FILE, 'r') as f:
    project_config = json.load(f)
    project_config.pop('_comment', None)
    project_config.pop('_note', None)
    f.close()

In [ ]:
np.random.seed(31071967)

#[Download dataset from Kaggle](https://www.kaggle.com/datasets/joebeachcapital/30000-spotify-songs?select=spotify_songs.csv)

In [ ]:
#kaggle_api_key = '{"username":"guyrayov","key":"54ae946c7dc7d70618ebf49de4b74dd4"}'

# STEP 1: checking if the dataset exist
import os
if not os.path.exists(DATA_FILE):

  print("dowloading Spotify dataset from kaggle....")

  # STEP 2: create API key file: ~/.kaggle/kaggle.json
  !pip install kaggle --quiet
  !mkdir -p ~/.kaggle
  !echo '{"username":"guyrayov","key":"54ae946c7dc7d70618ebf49de4b74dd4"}' > ~/.kaggle/kaggle.json
  !chmod 600 ~/.kaggle/kaggle.json

  # STEP 3: Download the dataset from Kaggle
  !kaggle datasets download -d joebeachcapital/30000-spotify-songs

  # STEP 4: Unzip the dataset
  !unzip -o 30000-spotify-songs.zip -d spotify_data

  # STEP 5: Check what files are inside. remove the ZIP file
  !ls spotify_data

  # STEP 6: remove the ZIP file
  !rm -rf 30000-spotify-songs.zip

else: print("Spotify dataset already exists.")

#Upload dataset with Pandas

In [ ]:
import pandas as pd
df = pd.read_csv(DATA_FILE)
display(df.head(1)), display(df.tail(1))

In [ ]:
# # appearnatly, five tracks appaers twice (different track_ids), each time in a different playlist. Even popularity is not identical.
# # decition: keep both copies
# duplicates = (df.groupby(['track_name', 'track_artist', 'track_album_id']).filter(lambda x: x['track_id'].nunique() > 1))
# duplicates = duplicates.sort_values(['track_artist', 'track_name', 'track_album_id'])

In [ ]:
unique_id_cols = ['track_name']
dup_cols       = ['track_album_name', 'playlist_name','playlist_genre','instrumentalness']
large_cat_cols = ['track_artist', 'track_album_id', 'playlist_id']
small_cat_cols = ['mode','key', 'playlist_subgenre']
cont_cols      = ['acousticness', 'danceability','duration_ms','energy', 'liveness', 'loudness', 'speechiness', 'tempo','valence']
date_cols      = ['release_date']
cat_cols       = large_cat_cols + small_cat_cols
y_col          = 'track_popularity'
MERGE_ON_COL   = 'track_id' # if we'll need to merge pickels later

#Flatfile

In [ ]:
def write_flat_file(df, filename):
  writer = pd.ExcelWriter(filename, engine="openpyxl")

  df.head().to_excel(writer, sheet_name='head')
  df.tail().to_excel(writer, sheet_name='tail')
  df.describe().to_excel(writer, sheet_name='describe')
  df.dtypes.to_excel(writer, sheet_name='data_type')
  df.select_dtypes(include=np.number).max().to_excel(writer, sheet_name='max - numeric columns')
  df.select_dtypes(include=np.number).min().to_excel(writer, sheet_name='min - numeric columns')
  df.isnull().sum(axis=0).to_excel(writer, sheet_name='NA')
  df.nunique().to_excel(writer, sheet_name='unique')

  writer.close()
  print(f"Flat file: {filename}")

flat_file = "spotify_flatfile.xlsx"
write_flat_file(df, f"{project_config['project_path']}{project_config['pickles_directory']}{flat_file}")

In [ ]:
if project_config['split_df'] == '1':
  from sklearn.model_selection import train_test_split

  # split to avoid data leakage
  train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

  # keep the name df for code compliaency
  df = train_df
  del train_df

  # just a mark
  df['set']  = 'train'
  test_df['set']  = 'test'

else:
  test_df = None

In [ ]:
def pickle_col(df, col='all', drop_col=False, include_merge_ID=True, pickle_name=""):

  import pickle

  if pickle_name == "":
    file_name = f"{project_config['project_path']}{project_config['pickles_directory']}{col}.pkl"
  else:
    file_name = f"{project_config['project_path']}{project_config['pickles_directory']}{pickle_name}.pkl"

  with open(file_name, 'wb') as f:

    if col =='all':
      pickle.dump(df, f)

    elif col in df.columns: # in case we aready droped the col before

      # track_id for a later merge, if we need.
      #and y_col so can can invetigate the pickel later indepandantly from the main df
      pickle.dump(df[[MERGE_ON_COL, col, y_col]], f)

    f.close()

    if drop_col == True:
      df.drop(col, axis=1, inplace=True, errors='ignore')

  if project_config['split_df'] == '1':
    with open(file_name+".test.pkl", 'wb') as f:

      if col =='all':
        pickle.dump(test_df, f)

      elif col in test_df.columns: # in case we aready droped the col before
        pickle.dump(test_df[[MERGE_ON_COL, col, y_col]], f)

      f.close()

    if drop_col == True:
      test_df.drop(col, axis=1, inplace=True, errors='ignore')

  return df

In [ ]:
def apply_cat_mean_as_target_encode(train_df, cat_cols, encode_cols, target_col, test_df=None):

  for i in range(len(cat_cols)):
    train_means = train_df.groupby(cat_cols[i])[target_col].mean()
    train_df[encode_cols[i]] = train_df[cat_cols[i]].map(train_means).astype(float)

    if test_df is not None:
        test_means = test_df.groupby(cat_cols[i])[target_col].mean()
        test_df[encode_cols[i]] = test_df[cat_cols[i]].map(test_means).astype(float)
      #test_df[encode_cols[i]] = test_df[cat_cols[i]].map(train_means).astype(float)
      #test_df[encode_cols[i]].fillna(train_means, inplace=True)

# Dates

In [ ]:
#track_album_release_date
if "track_album_release_date" in df.columns: # maybe I'm just rerunnig this cell


  df.track_album_release_date = pd.to_datetime(df.track_album_release_date, format='mixed')
  df['release_month' ] = df['track_album_release_date'].dt.month.astype(int)
  df['release_year'  ] = df['track_album_release_date'].dt.year.astype(int)
  df['release_decade'] = ((df['release_year'] // 10) * 10).astype(int)

  if project_config['split_df'] == '1':

    test_df.track_album_release_date = pd.to_datetime(test_df.track_album_release_date, format='mixed')
    test_df['release_month' ] = test_df['track_album_release_date'].dt.month.astype(int)
    test_df['release_year'  ] = test_df['track_album_release_date'].dt.year.astype(int)
    test_df['release_decade'] = ((test_df['release_year'] // 10) * 10).astype(int)

  apply_cat_mean_as_target_encode(
      df,
      cat_cols=['release_decade', 'release_month','release_year'],
      encode_cols=['release_decade_mean_popularity','release_month_mean_popularity','release_year_mean_popularity'],
      target_col='track_popularity',
      test_df=test_df)

  pickle_col(df, col="track_album_release_date", drop_col=True)

In [ ]:
plot_columns = 3
plot_rows = 2

fig = plt.figure(figsize=(12,12))
plt.subplots_adjust(hspace=1.0)
sb.set(font_scale=0.8)

for plot_counter, col in enumerate(['release_decade','release_year', 'release_month'],start=1):
    plt.subplot(plot_rows, plot_columns, plot_counter)
    sb.histplot(x=col, data=df)
plt.show()

fig = plt.figure(figsize=(12,12))
plt.subplots_adjust(hspace=1.0)
sb.set(font_scale=0.8)

for plot_counter, col in enumerate(['release_decade','release_year', 'release_month'],start=1):
    plt.subplot(plot_rows, plot_columns, plot_counter)
    sb.barplot(x=col, y=f"{col}_mean_popularity", data=df)

plt.show()

# Categories

Uniqe IDs: **drop** to a pickel. Keeping track_id col with each pickel for possible merges later

Duplications (album_name, album_id, genre): **drop** to a pickel

Lerge cat cols: **replace** with the 'mean' popularity of each category - making it a continuence variabe on

Small cat cols: **group** categories based on ANOVA-tests between mean popoularity of each cat. **Replace** with with the 'mean' popularity of each category.

In [ ]:
# unique ids and duplicaitons: drop
for col in unique_id_cols + dup_cols:
  df = pickle_col(df, col=col, drop_col=True)

In [ ]:
# musical mode
df['mode'] = df['mode'].map({0: "Minor", 1: "Major"}).astype('string')

if project_config['split_df'] == '1':
  test_df['mode'] = test_df['mode'].map({0: "Minor", 1: "Major"}).astype('string')

In [ ]:
# 12 categories for musical keys is small enoght to keep
# Mapping key-numbers into musical symbols, simpply for a better visualizalizaiton

musical_key_dict = {
    0: "C",
    1: "C♯/D♭",
    2: "D",
    3: "D♯/E♭",
    4: "E",
    5: "F",
    6: "F♯/G♭",
    7: "G",
    8: "G♯/A♭",
    9: "A",
    10: "A♯/B♭",
    11: "B"
}
if df['key'].dtype == 'int64': # or maybe I'm just rerunnig this cell

  df['key'] = df['key'].map(musical_key_dict).astype('string')

  if project_config['split_df'] == '1':
    test_df['key'] = test_df['key'].map(musical_key_dict).astype('string')


In [ ]:
# using ANOVA for grouping 24 musical genres into 9

genre_map = {

    'post-teen pop'     : 'Teen Pop',
    'permanent wave'    : 'New Wave',

    'hip pop'           : 'Mainstream Pop & Hip-Hop',
    'hip hop'           : 'Mainstream Pop & Hip-Hop',
    'dance pop'         : 'Mainstream Pop & Hip-Hop',
    'reggaeton'         : 'Mainstream Pop & Hip-Hop',

    'latin pop'         : 'Latin & Urban Trap',
    'urban contemporary': 'Latin & Urban Trap',
    'trap'              : 'Latin & Urban Trap',

    'pop edm'           : 'Electro/Latin Pop Fusion',
    'latin hip hop'     : 'Electro/Latin Pop Fusion',
    'tropical'          : 'Electro/Latin Pop Fusion',
    'electropop'        : 'Electro/Latin Pop Fusion',
    'indie poptimism'   : 'Electro/Latin Pop Fusion',

    'classic rock'      : 'Classic Rock',
    'album rock'        : 'Classic Rock',

    'southern hip hop'  : 'Hardcore Hip-Hop & Rock/EDM',
    'hard rock'         : 'Hardcore Hip-Hop & Rock/EDM',
    'electro house'     : 'Hardcore Hip-Hop & Rock/EDM',
    'gangster rap'      : 'Hardcore Hip-Hop & Rock/EDM',

    'neo soul'          : 'Soul & Big Room EDM',
    'big room'          : 'Soul & Big Room EDM',

    'new jack swing'    : 'Retro Swing & Prog House',
    'progressive electro house' : 'Retro Swing & Prog House'
}

df['playlist_genre_grouped'] = df['playlist_subgenre'].map(genre_map).astype('string')

if project_config['split_df'] == '1':
  test_df['playlist_genre_grouped'] = test_df['playlist_subgenre'].map(genre_map).astype('string')

small_cat_cols.append('playlist_genre_grouped')
cat_cols.append('playlist_genre_grouped')

small_cat_cols.remove('playlist_subgenre')
cat_cols.remove('playlist_subgenre')
pickle_col(df, col='playlist_subgenre', drop_col=True)

In [ ]:
# target encoding: replace categories with mean_popularity of each category
for col in cat_cols:

  if col in df.columns: # maybe I'm just reruning the cell

    apply_cat_mean_as_target_encode(
        df,
        cat_cols=[col],
        encode_cols=[f'{col}_mean_popularity'],
        target_col='track_popularity',
        test_df=test_df)

#keep original col for visuaizations later, it will not be used for the train

In [ ]:
# plot categories mean popularity

plot_columns = 3
n_plot = len(large_cat_cols) + len(small_cat_cols) + len(cat_cols)
plot_rows = n_plot//plot_columns + n_plot%plot_columns

fig = plt.figure(figsize=(20,28))
plt.subplots_adjust(hspace=1.0)
sb.set(font_scale=0.8)

for plot_counter, col in enumerate(large_cat_cols, start=1):
  plt.subplot(plot_rows, plot_columns, plot_counter)
  sb.histplot(x=f"{col}_mean_popularity", data=df, kde=True, bins=15)
  # pickle_col(df, col=col, drop_col=True)  # we dont need the original col anymore

for plot_counter, col in enumerate(small_cat_cols, start=1):
    plt.subplot(plot_rows, plot_columns, len(large_cat_cols) + plot_counter)
    order = df.groupby(col)[f"{col}_mean_popularity"].mean().sort_values(ascending=False).index
    sb.barplot(x=col, y=f"{col}_mean_popularity", data=df, order=order)
    plt.xticks(rotation=85)
    #keep orirignal cols for eda visualzations

for plot_counter, col in enumerate(small_cat_cols, start=1):
    plt.subplot(plot_rows, plot_columns, len(large_cat_cols) + len(small_cat_cols) + plot_counter)
    sb.countplot(x=col, data=df)
    plt.xticks(rotation=85)
    #keep orirignal cols for eda visualzations

plt.show()

# Continuance

In [ ]:
# plot values
plot_columns = 3
plot_rows = (len(cont_cols)//plot_columns) + len(cont_cols)%plot_columns

fig = plt.figure(figsize=(20,15))
plt.subplots_adjust(hspace=0.8)
sb.set(font_scale=1.2)

for plot_counter, col in enumerate(cont_cols, start=1):
    plt.subplot(plot_rows, plot_columns, plot_counter)
    sb.histplot(x=col, data=df, kde=True)

plt.show()

# Pickle

In [ ]:
from google.colab import files
import os

with open(f"{project_config['project_path']}{project_config['pickles_directory']}{project_config['pickle1']}", 'wb') as f:
  pickle.dump(df, f)
  f.close()

if project_config['split_df'] == '1':
  with open(f"{project_config['project_path']}{project_config['pickles_directory']}{project_config['pickle1_test']}", 'wb') as f:
    pickle.dump(test_df, f)
    f.close()